## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [4]:
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical

In [8]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [16]:
data_generator = ImageDataGenerator(rotation_range=30,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest',
                                    rescale=1./255)

data_generator.fit(x_train)

In [6]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [18]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit_generator(data_generator.flow(x_train, y_train, shuffle=False,batch_size=batch_size),
                              steps_per_epoch=int(len(x_train)/batch_size),
                              epochs=epochs,
                              verbose=1,
                              validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/30
781/781 [==============================] - 804s 1s/step - loss: 2.0430 - acc: 0.4286 - val_loss: 14.5218 - val_acc: 0.1180
Epoch 2/30
781/781 [==============================] - 807s 1s/step - loss: 1.6495 - acc: 0.5319 - val_loss: 13.9400 - val_acc: 0.1528
Epoch 3/30
781/781 [==============================] - 795s 1s/step - loss: 1.4685 - acc: 0.5827 - val_loss: 14.6155 - val_acc: 0.1103
Epoch 4/30
781/781 [==============================] - 791s 1s/step - loss: 1.3491 - acc: 0.6178 - val_loss: 14.0348 - val_acc: 0.1313
Epoch 5/30
781/781 [==============================] - 793s 1s/step - loss: 1.2651 - acc: 0.6462 - val_loss: 14.7063 - val_acc: 0.1021
Epoch 6/30
781/781 [==============================] - 795s 1s/step - loss: 1.1943 - acc: 0.6699 - val_loss: 14.4752 - val_acc: 0.1142
Epoch 7/30
781/781 [==============================] - 796s 1s/step - loss: 1.1398 - acc: 0.6901 - val_loss: 14.0499 - val_acc: 0.1269
Epoch 8/30
781/781 [==============================] - 793s 1s/